# 제2회 통신망 안정성 확보를 위한 인공지능 해커톤
# 분야 1 :무선 기지국 장비의 통계 데이터를 활용한 인구 밀집도 예측  

- 출제 배경<br>
    : 많은 사람들이 특정 시간/장소에 모이면 무선 통신 품질의 저하와 함께 각종 사고 위험으로부터 안전할 수 없게 됩니다. 통신사업자의 무선 통신 기지국에서는 서비스를 제공하는 단말의 통계 정보를 통해 인구의 밀집 정도를 파악 가능합니다. 무선 네트워크의 성능을 안정적으로 유지하고 인구 밀집도를 완화하기 위해, AI를 활용하여 특정 축제 지역 내의 인구 수를 예측하고자 합니다.
- 학습 데이터<br>
    : 무선 기지국 장비인 RU(Radio Unit)에서는 서비스 지역인 셀 내에 존재하는 단말들의 통계 데이터를 관리합니다. 본 데이터에서는 업링크/다운링크 데이터 크기 및 블록 오류율(BLER), 셀 내 평균 수신 신호 강도(RSSI) 및 사용자 단말 수 등의 각종 통계 정보가 5분 단위로 제공됩니다.
- 문제 구성 및 풀이 요령<br>
    : 축제 지역 근방 10곳의 기지국에서 축제 기간 전후에 해당하는 모든 타임스탬프에 대해 인구 수를 예측하는 회귀 문제입니다. 다양한 데이터 컬럼 중 인구 수 변화에 유의미한 값을 선별하고 시계열 데이터의 주기성을 고려하는 것이 중요합니다. 인구 1인당 단말 1대를 가지고 있다고 가정합니다.
- 채점 기준<br>
    : 평균 절대 오차(MAE)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import RobustScaler

pd.options.display.max_columns = None
warnings.filterwarnings(action='ignore')

# 해당 경로는 저장 위치에 따라 변경
%cd '/content/drive/MyDrive/colab/제2회 통신망 안정성 확보를 위한 인공지능 해커톤/Q1'

Mounted at /content/drive
/content/drive/MyDrive/colab/제2회 통신망 안정성 확보를 위한 인공지능 해커톤/Q1


# 공통 전처리

## 전처리 함수 생성

In [ ]:
# 결측치 선형 보간 함수
def func_interpolate(df):
    for col in ['rlculbyte', 'rlcdlbyte', 'totprbulavg', 'totprbdlavg', 'dlreceivedriavg', 'dltransmittedmcsavg', 'airmaculbyte', 'airmacdlbyte', 'bler_ul', 'bler_dl', 'rachpreamblea', 'numrar', 'nummsg3', 'attpaging', 'rssipathavg', 'dlreceivedcqiavg']:
        df_tmp = df[['datetime',col]].set_index('datetime')
        df[col] = df_tmp.interpolate(method='values')[col].tolist()

# log scaling 함수
def log_scale(data,col):
    data[col] = np.log1p(data[col]) # 0인 경우를 방지하기 위해 +1 해줌

# robust scaling 함수
def rs(data,col):
    scaler = RobustScaler()
    data[col] = scaler.fit_transform(data[[col]])

# standard scailing 함수
def standard_scale(data,col):
    mean = np.mean(data[col])
    std = np.std(data[col])
    data[col] = (data[col] - mean) / std

# 이상치 제거 함수, 1.5IQR을 벗어나는 값을 1.5IQR로 대체
def remov_outlier(data,col) :
    # IQR 계산
    q1 = np.percentile(data[col], 25)
    q3 = np.percentile(data[col], 75)
    iqr = q3 - q1
    # IQR 기준으로 이상치 제거
    upper_bound = q3 + 1.5 * iqr
    if upper_bound != 0 :
        data[col] = data.loc[data[col]<=upper_bound,col]

# 사용할 컬럼 설정
use_col = ['scgfail','erabaddatt', 'erabaddsucc', 'endcaddatt', 'endcaddsucc', 'endcmodbymenbatt', 'endcmodbymenbsucc', 'endcmodbysgnbatt', 'endcmodbysgnbsucc', 'connestabatt', 'connestabsucc', 'redirectiontolte_coverageout', 'redirectiontolte_epsfallback', 'handoveratt', 'handoversucc', 'reestabatt', 'reestabsucc', 'rlculbyte', 'rlcdlbyte', 'totprbulavg', 'totprbdlavg', 'dlreceivedriavg', 'airmaculbyte', 'airmacdlbyte', 'rachpreamblea', 'numrar', 'nummsg3', 'attpaging', 'rssipathavg', 'dlreceivedcqiavg', 'endcrelbymenb']
log_col = ['scgfail', 'erabaddatt', 'erabaddsucc', 'endcaddatt', 'endcaddsucc', 'endcmodbymenbatt', 'endcmodbymenbatt', 'endcmodbymenbsucc', 'endcmodbysgnbatt', 'endcmodbysgnbsucc', 'connestabatt', 'connestabsucc', 'redirectiontolte_coverageout', 'redirectiontolte_epsfallback', 'handoveratt', 'handoversucc', 'rlculbyte', 'rlcdlbyte', 'totprbulavg', 'totprbdlavg', 'airmaculbyte', 'airmacdlbyte', 'rachpreamblea', 'numrar', 'nummsg3', 'endcrelbymenb']

# train set load 및 공통 전처리 실행 함수
def make_train_set(path='data/Q1_train.csv'):
    # data load
    train = pd.read_csv(path)

    # datetime datatype 변경
    train['datetime'] = pd.to_datetime(train['datetime'])

    # train, target split
    target = 'uenomax'
    x_train = train.drop(target, axis=1)
    y_train = train[['ru_id',target]]

    # 기지국별로 데이터 분리
    x_train_A = x_train.loc[x_train['ru_id']=='BaseStationA']
    x_train_C = x_train.loc[x_train['ru_id']=='BaseStationC']
    x_train_D = x_train.loc[x_train['ru_id']=='BaseStationD']
    x_train_E = x_train.loc[x_train['ru_id']=='BaseStationE']
    x_train_F = x_train.loc[x_train['ru_id']=='BaseStationF']
    x_train_G = x_train.loc[x_train['ru_id']=='BaseStationG']
    x_train_H = x_train.loc[x_train['ru_id']=='BaseStationH']
    x_train_I = x_train.loc[x_train['ru_id']=='BaseStationI']

    y_train_A = y_train.loc[y_train['ru_id']=='BaseStationA']
    y_train_C = y_train.loc[y_train['ru_id']=='BaseStationC']
    y_train_D = y_train.loc[y_train['ru_id']=='BaseStationD']
    y_train_E = y_train.loc[y_train['ru_id']=='BaseStationE']
    y_train_F = y_train.loc[y_train['ru_id']=='BaseStationF']
    y_train_G = y_train.loc[y_train['ru_id']=='BaseStationG']
    y_train_H = y_train.loc[y_train['ru_id']=='BaseStationH']
    y_train_I = y_train.loc[y_train['ru_id']=='BaseStationI']

    # 결측치 제거(선형 보간법으로 채워줌)
    df_nan = [x_train_E, x_train_F, x_train_G]
    for df in df_nan:
        func_interpolate(df)
    return x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I

# test set load 및 공통 전처리 실행 함수
def make_test_set(path='data/Q1_test.csv'):
    # data load
    x_test = pd.read_csv(path)

    # datetime datatype 변경
    x_test['datetime'] = pd.to_datetime(x_test['datetime'])

    # 기지국별로 데이터 분리
    x_test_J = x_test.loc[x_test['ru_id']=='BaseStationJ']
    x_test_B = x_test.loc[x_test['ru_id']=='BaseStationB']
    return x_test_J,x_test_B

# Submit 1 : 상관관계 높은 컬럼 사용 / 로그 스케일링 / 이상치 제거 / 정규화

## 01) train set 생성

In [ ]:
# 공통전처리가 끝난 train set 불러오기
x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I = make_train_set()

# 상관관계가 낮은 변수 제거
x_train_A = x_train_A[['datetime',	'ru_id']+use_col]
x_train_C = x_train_C[['datetime',	'ru_id']+use_col]
x_train_D = x_train_D[['datetime',	'ru_id']+use_col]
x_train_E = x_train_E[['datetime',	'ru_id']+use_col]
x_train_F = x_train_F[['datetime',	'ru_id']+use_col]
x_train_G = x_train_G[['datetime',	'ru_id']+use_col]
x_train_H = x_train_H[['datetime',	'ru_id']+use_col]
x_train_I = x_train_I[['datetime',	'ru_id']+use_col]

xs = [x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I]
ys = [y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I]

# train set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 이상치 제거 및 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            remov_outlier(df,col)
            standard_scale(df,col)

# train data 합치기
x_train_merge = x_train_A.copy()
y_train_merge = y_train_A.copy()
for i in range(1,8):
    x_train_merge = pd.concat([x_train_merge, xs[i]], ignore_index=False)
    y_train_merge = pd.concat([y_train_merge, ys[i]], ignore_index=False)

# index 순으로 정렬
x_train_merge.sort_index(inplace=True)
y_train_merge.sort_index(inplace=True)

# AutoML input data type으로 변경
train_merge = pd.concat([x_train_merge, y_train_merge[['uenomax']]], axis=1)

# datetime, ru_id 제거
train_merge_f = train_merge.drop(columns=['datetime','ru_id'])

# 결측치가 있는 행 제거
train_merge_f = train_merge_f.dropna()

In [ ]:
train_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,-0.650928,-0.832874,-0.832691,-0.096652,-0.096512,-1.790962,-1.552873,-3.350626,-3.353591,-0.268361,-0.264663,-0.418631,-0.324742,-1.738269,-1.735410,0,0,-3.713571,-3.834820,-1.315288,-1.154916,-0.524373,-2.750116,-2.937247,-1.066092,-1.066092,-1.201694,-0.975277,-1.160101,-0.817691,-1.330158,1
1,-0.657152,0.324175,0.324511,0.682312,0.682469,1.310287,1.618175,1.100066,1.077706,-0.986157,-0.983742,-0.511178,-0.235394,-0.984750,-0.984162,0,0,0.255654,0.472608,0.033991,0.222232,-0.949112,-0.033831,0.489980,1.216417,1.216417,0.368558,-0.976752,-0.626002,-0.573283,1.419193,5
2,1.030015,-0.426198,-0.425954,-0.424311,-0.424243,0.612832,0.631136,-0.628696,-0.615033,-2.072921,-2.070899,1.000958,-0.479227,-1.311945,-1.309987,0,0,0.631021,0.388912,0.515251,-0.045995,1.262533,0.700004,0.306421,-0.268890,-0.268890,-0.446870,-1.058652,-1.035583,-0.283147,-0.633761,6
3,-0.406465,-1.114238,-1.113994,-0.976695,-0.976614,-0.826929,-0.781595,-1.818113,-1.813472,1.005675,1.008167,-0.312775,-0.183432,-0.688403,-0.687824,0,0,-0.487264,-0.623035,-0.544307,-0.590302,-0.708825,-0.599738,-0.756642,2.114622,2.114622,-0.438913,-0.859361,-2.259611,0.915922,-0.575646,1
4,-0.561473,0.803532,0.803636,0.974515,0.974593,0.593810,0.596539,0.709139,0.721250,-0.815109,-0.813346,-0.246645,-0.207081,0.780193,0.780557,0,0,0.306237,0.848377,-0.019062,0.465122,-0.507213,0.144306,0.884744,0.554075,0.554075,0.729070,-1.020477,-1.546140,1.869399,0.963641,4


In [ ]:
train_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113432 entries, 0 to 137443
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   scgfail                       113432 non-null  float64
 1   erabaddatt                    113432 non-null  float64
 2   erabaddsucc                   113432 non-null  float64
 3   endcaddatt                    113432 non-null  float64
 4   endcaddsucc                   113432 non-null  float64
 5   endcmodbymenbatt              113432 non-null  float64
 6   endcmodbymenbsucc             113432 non-null  float64
 7   endcmodbysgnbatt              113432 non-null  float64
 8   endcmodbysgnbsucc             113432 non-null  float64
 9   connestabatt                  113432 non-null  float64
 10  connestabsucc                 113432 non-null  float64
 11  redirectiontolte_coverageout  113432 non-null  float64
 12  redirectiontolte_epsfallback  113432 non-nul

In [ ]:
train_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000,113432.000000
mean,-0.047554,-0.086934,-0.086933,-0.078407,-0.078410,-0.093556,-0.075921,-0.110897,-0.110880,-0.068810,-0.068769,-0.046468,-0.038997,-0.054586,-0.054802,0.016371,0.015631,-0.148139,-0.131739,-0.107295,-0.076968,-0.023558,-0.167069,-0.146389,-0.091461,-0.091461,-0.101964,-0.057110,-0.068405,-0.008171,-0.069181,3.465565
std,0.928451,0.985324,0.985332,0.984180,0.984183,0.964242,0.967291,0.978605,0.978547,0.968184,0.968127,0.932680,0.921951,0.974332,0.974144,0.126898,0.124042,0.961236,0.987719,0.927633,0.953715,1.013750,0.954525,0.987583,0.987588,0.987588,0.981657,1.008517,0.966603,1.021051,0.978916,2.278901
min,-0.670763,-5.586383,-5.586476,-4.761558,-4.761491,-3.019025,-2.438404,-4.331944,-4.328878,-4.631087,-4.630437,-0.655573,-0.479227,-2.106024,-2.105437,0.000000,0.000000,-5.263894,-5.588878,-1.592566,-1.379072,-7.308796,-6.215599,-5.751181,-8.898089,-8.898089,-7.307354,-1.818940,-10.706978,-12.605842,-4.131801,0.000000
25%,-0.650928,-0.638906,-0.638659,-0.600304,-0.600164,-0.826929,-0.781595,-0.628696,-0.645162,-0.955204,-0.954822,-0.434171,-0.310128,-0.688403,-0.687824,0.000000,0.000000,-0.644424,-0.556129,-0.774911,-0.861240,-0.693214,-0.674420,-0.593546,-0.693376,-0.693376,-0.662305,-1.081080,-0.720673,-0.541147,-0.621400,2.000000
50%,-0.521983,0.088195,0.088518,0.071144,0.071293,0.066539,-0.138303,0.069910,0.075948,-0.026636,-0.025442,-0.316598,-0.235394,-0.440922,-0.439731,0.000000,0.000000,0.086923,0.165688,-0.387238,-0.348689,0.011442,0.036748,0.130899,0.044057,0.044057,0.078557,-0.052685,-0.203939,0.089148,-0.065645,3.000000
75%,-0.251113,0.623170,0.623521,0.624261,0.624353,0.653964,0.602314,0.580224,0.580018,0.649093,0.651337,-0.246645,-0.183432,0.780193,0.780557,0.000000,0.000000,0.550513,0.570061,0.515251,0.465122,0.648011,0.526608,0.560073,0.643494,0.643494,0.603689,0.890955,0.453272,0.659908,0.649893,5.000000
max,10.299908,2.372415,2.372502,2.495759,2.495918,2.204752,3.573739,2.580503,2.592201,2.971762,2.973958,10.983809,15.842499,6.377795,6.396067,1.000000,1.000000,1.892538,1.804699,3.412479,2.985575,2.777282,1.953005,1.966060,2.924786,2.924786,2.477482,2.525293,3.199458,2.571111,3.712982,18.000000


## 02) test set 생성

In [ ]:
# 공통전처리가 끝난 test set 불러오기
x_test_J,x_test_B=make_test_set()

# 상관관계가 낮은 변수 제거
x_test_J = x_test_J[['datetime','ru_id']+use_col]
x_test_B = x_test_B[['datetime','ru_id']+use_col]

xs = [x_test_J,x_test_B]

# test set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            standard_scale(df,col)

# train data 합치기
x_test_merge = pd.concat([x_test_J,x_test_B], ignore_index=False)

# index 순으로 정렬
x_test_merge.sort_index(inplace=True)

# datetime, ru_id 제거
x_test_merge_f = x_test_merge.drop(columns=['datetime','ru_id'])

In [ ]:
x_test_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,-0.667542,-0.481082,-0.480596,-0.257629,-0.257425,-0.718056,-0.670180,-1.382757,-1.379720,0.010077,0.011238,-0.226592,-0.171584,1.982971,1.987162,0,0,-1.562210,-1.751742,-0.410825,-0.523185,-1.070125,-0.602207,-0.271383,0.375598,0.375598,-0.332304,-0.859361,-0.318702,-0.011695,-0.577303
1,-0.427040,-1.454413,-1.453854,-1.349899,-1.349338,-1.000441,-0.916495,-1.413417,-1.410143,-0.469494,-0.468596,-0.128414,-0.086296,-0.447929,-0.447758,0,0,-1.509133,-1.624831,-0.469801,-0.633370,0.278680,-1.920447,-1.815504,-1.467930,-1.467930,-1.201107,-1.020477,-0.298595,-1.559757,-1.002757
2,-0.667542,-1.004315,-1.003808,-1.144738,-1.144495,-0.718056,-0.670180,-0.328479,-0.324267,0.010077,0.011238,-0.226592,-0.171584,-0.323645,-0.322739,0,0,0.529450,-0.234396,-0.410825,-0.523185,2.216891,0.324410,-0.267118,-0.652779,-0.652779,0.489501,-0.919680,-0.415243,0.862119,-0.577303
3,-0.427040,0.447827,0.448464,0.530739,0.531318,-1.000441,-0.916495,0.464955,0.470439,-0.469494,-0.468596,-0.128414,-0.086296,-0.447929,-0.447758,0,0,0.132145,-0.138808,-0.469801,-0.633370,-0.835845,-0.060003,-0.204914,0.089064,0.089064,0.278165,-1.084683,-0.298595,-0.750170,0.398415
4,-0.667542,-0.716359,-0.715864,-0.490970,-0.490755,-0.718056,-0.670180,-0.916231,-0.912675,0.573040,0.574515,-0.226592,-0.171584,-0.323645,-0.322739,0,0,0.251629,-0.391863,-0.410825,-0.523185,-0.193587,-0.107811,-0.508853,-0.065758,-0.065758,-0.506874,-0.984066,-0.415243,0.586177,-0.577303


In [ ]:
x_test_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34362 entries, 0 to 34361
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scgfail                       34362 non-null  float64
 1   erabaddatt                    34362 non-null  float64
 2   erabaddsucc                   34362 non-null  float64
 3   endcaddatt                    34362 non-null  float64
 4   endcaddsucc                   34362 non-null  float64
 5   endcmodbymenbatt              34362 non-null  float64
 6   endcmodbymenbsucc             34362 non-null  float64
 7   endcmodbysgnbatt              34362 non-null  float64
 8   endcmodbysgnbsucc             34362 non-null  float64
 9   connestabatt                  34362 non-null  float64
 10  connestabsucc                 34362 non-null  float64
 11  redirectiontolte_coverageout  34362 non-null  float64
 12  redirectiontolte_epsfallback  34362 non-null  float64
 13  h

In [ ]:
x_test_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
count,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,34362.000000,3.436200e+04,3.436200e+04,34362.000000,34362.000000,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04
mean,1.654252e-17,-2.365581e-16,1.108349e-16,4.218343e-16,-1.157977e-16,-2.440022e-17,3.432574e-17,-3.639355e-17,2.646804e-17,2.481379e-17,-1.075264e-16,-7.444136e-18,0.000000,-3.308505e-17,2.729516e-17,0.002212,0.002066,-1.162112e-16,-1.852763e-16,3.391217e-17,1.687337e-16,-1.819678e-17,2.770873e-17,4.962757e-18,-3.688983e-16,-3.688983e-16,4.301056e-17,-1.464013e-16,6.663742e-15,-3.639355e-16,4.135631e-19
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015,1.000015e+00,1.000015e+00,0.046978,0.045410,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-6.675420e-01,-3.276950e+00,-3.276468e+00,-3.151740e+00,-3.151197e+00,-1.000441e+00,-9.164945e-01,-2.753031e+00,-2.751333e+00,-9.523143e-01,-9.516885e-01,-2.265920e-01,-0.171584,-4.479294e-01,-4.477579e-01,0.000000,0.000000,-1.562210e+00,-1.803986e+00,-4.698012e-01,-4.716004e+00,-4.138005e+00,-2.451641e+00,-2.052487e+00,-5.315769e+00,-5.315769e+00,-3.146559e+00,-1.930282e+00,-4.152431e-01,-4.744131e+00,-3.055436e+00
25%,-6.675420e-01,-5.431438e-01,-5.425478e-01,-6.225366e-01,-6.219684e-01,-7.180562e-01,-6.701800e-01,-5.852263e-01,-5.813007e-01,-4.694943e-01,-4.685963e-01,-2.265920e-01,-0.171584,-4.479294e-01,-4.477579e-01,0.000000,0.000000,-9.010179e-01,-7.127420e-01,-4.698012e-01,-5.231848e-01,-4.127215e-01,-6.868895e-01,-7.110441e-01,-5.715427e-01,-5.715427e-01,-5.068745e-01,-1.017275e+00,-1.256198e-01,-5.635772e-01,-5.773034e-01
50%,-4.270402e-01,9.514503e-02,9.576755e-02,1.105541e-01,1.107427e-01,-7.180562e-01,-6.701800e-01,1.708896e-01,1.760273e-01,-4.694943e-01,-4.685963e-01,-1.284139e-01,-0.086296,-3.236450e-01,-3.227385e-01,0.000000,0.000000,2.034653e-01,3.375259e-01,-4.108253e-01,-2.578315e-01,5.577489e-02,1.430597e-01,3.168229e-01,1.521601e-01,1.521601e-01,1.977770e-01,4.934158e-02,-2.907865e-02,3.429522e-02,-4.033132e-01
75%,5.177369e-01,7.256316e-01,7.017859e-01,7.319203e-01,7.320817e-01,8.109877e-01,6.581299e-01,7.017032e-01,7.074654e-01,5.730401e-01,5.745145e-01,-1.284139e-01,-0.086296,-3.236450e-01,-3.227385e-01,0.000000,0.000000,8.219134e-01,7.888894e-01,3.655041e-01,1.314463e-01,5.015848e-01,7.736888e-01,7.812339e-01,7.366734e-01,7.366734e-01,6.950006e-01,9.170064e-01,6.746246e-02,6.321676e-01,7.653256e-01
max,1.583369e+01,4.868842e+00,4.869115e+00,4.416785e+00,4.417402e+00,3.844610e+00,6.652412e+00,3.356580e+00,3.346015e+00,5.565503e+00,5.578637e+00,2.399095e+01,12.548270,8.211848e+00,8.224908e+00,1.000000,1.000000,3.985909e+00,2.177397e+00,3.822642e+01,1.724130e+01,4.959684e+00,4.264536e+00,2.300904e+00,5.013328e+00,5.013328e+00,3.958190e+00,2.525293e+00,9.008533e+01,3.351735e+00,7.752723e+00


## 03) data set 저장

In [ ]:
train_merge_f.to_csv('preprocessed_data/submit1_train.csv')
x_test_merge_f.to_csv('preprocessed_data/submit1_test.csv')

# Submit2 : 상관관계 높은 컬럼 사용 / 로그 스케일링 / 정규화

## 01) train set 생성

In [ ]:
# 공통전처리가 끝난 train set 불러오기
x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I = make_train_set()

# 상관관계가 낮은 변수 제거
x_train_A = x_train_A[['datetime',	'ru_id']+use_col]
x_train_C = x_train_C[['datetime',	'ru_id']+use_col]
x_train_D = x_train_D[['datetime',	'ru_id']+use_col]
x_train_E = x_train_E[['datetime',	'ru_id']+use_col]
x_train_F = x_train_F[['datetime',	'ru_id']+use_col]
x_train_G = x_train_G[['datetime',	'ru_id']+use_col]
x_train_H = x_train_H[['datetime',	'ru_id']+use_col]
x_train_I = x_train_I[['datetime',	'ru_id']+use_col]

xs = [x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I]
ys = [y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I]

# train set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            standard_scale(df,col)

# train data 합치기
x_train_merge = x_train_A.copy()
y_train_merge = y_train_A.copy()
for i in range(1,8):
    x_train_merge = pd.concat([x_train_merge, xs[i]], ignore_index=False)
    y_train_merge = pd.concat([y_train_merge, ys[i]], ignore_index=False)

# index 순으로 정렬
x_train_merge.sort_index(inplace=True)
y_train_merge.sort_index(inplace=True)

# AutoML input data type으로 변경
train_merge = pd.concat([x_train_merge, y_train_merge[['uenomax']]], axis=1)

# datetime, ru_id 제거
train_merge_f = train_merge.drop(columns=['datetime','ru_id'])

In [ ]:
train_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,-0.642055,-0.832913,-0.832730,-0.096805,-0.096665,-1.788567,-1.531903,-3.349084,-3.352038,-0.268580,-0.264883,-0.418631,-0.324742,-1.738269,-1.735410,0,0,-3.682557,-3.832811,-0.944992,-0.924894,-0.525700,-2.722273,-2.934820,-1.065644,-1.065644,-1.200997,-0.975277,-0.622757,-0.817923,-1.330158,1
1,-0.633315,0.323117,0.323452,0.680445,0.680602,1.310287,1.606385,1.097706,1.074189,-0.986157,-0.983742,-0.511178,-0.235394,-0.984477,-0.983888,0,0,0.247800,0.469808,-0.118880,-0.005310,-0.940073,-0.043042,0.484373,1.216417,1.216417,0.368558,-0.976752,-0.299750,-0.574049,1.415736,5
2,0.992809,-0.426198,-0.425954,-0.424311,-0.424243,0.612405,0.629271,-0.628696,-0.615033,-2.072355,-2.070336,0.980891,-0.479227,-1.311945,-1.309987,0,0,0.581540,0.386094,0.245136,-0.099470,1.250790,0.642649,0.303265,-0.269324,-0.269324,-0.446870,-1.058652,-0.695473,-0.291506,-0.633810,6
3,-0.406465,-1.114238,-1.113994,-0.976695,-0.976614,-0.826728,-0.778351,-1.817508,-1.812862,0.996785,0.999245,-0.312775,-0.183432,-0.684746,-0.684214,0,0,-0.487264,-0.623035,-0.454600,-0.442444,-0.718960,-0.599815,-0.756642,2.061341,2.061341,-0.438986,-0.859361,-0.439955,0.915922,-0.575182,1
4,-0.556887,0.803532,0.803636,0.974515,0.974593,0.586366,0.370437,0.708844,0.720950,-0.815109,-0.813346,-0.246645,-0.207081,0.613963,0.615916,0,0,0.304983,0.848377,-0.111756,0.135980,-0.511826,0.139361,0.884744,0.542622,0.542622,0.723794,-1.020477,-0.463952,1.735165,0.963641,4


In [ ]:
train_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137445 entries, 0 to 137444
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   scgfail                       137445 non-null  float64
 1   erabaddatt                    137445 non-null  float64
 2   erabaddsucc                   137445 non-null  float64
 3   endcaddatt                    137445 non-null  float64
 4   endcaddsucc                   137445 non-null  float64
 5   endcmodbymenbatt              137445 non-null  float64
 6   endcmodbymenbsucc             137445 non-null  float64
 7   endcmodbysgnbatt              137445 non-null  float64
 8   endcmodbysgnbsucc             137445 non-null  float64
 9   connestabatt                  137445 non-null  float64
 10  connestabsucc                 137445 non-null  float64
 11  redirectiontolte_coverageout  137445 non-null  float64
 12  redirectiontolte_epsfallback  137445 non-nul

In [ ]:
train_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,137445.000000,137445.00000,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,1.374450e+05,137445.000000
mean,8.271442e-17,1.406145e-17,-1.385467e-16,1.062880e-16,4.507936e-17,-9.429444e-17,-2.481433e-18,-1.488860e-16,-4.135721e-18,1.323431e-16,8.933158e-17,4.135721e-18,-2.770933e-17,-2.274647e-18,1.271734e-16,0.018014,0.01696,-1.654288e-18,-1.963434e-16,-8.271442e-18,-3.655977e-16,1.290345e-16,-2.076132e-16,-6.865297e-17,-1.670831e-16,-1.670831e-16,-2.117489e-16,7.857870e-18,6.397961e-16,-2.295325e-16,4.466579e-17,3.718826
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,0.133004,0.12912,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,2.781951
min,-6.659913e-01,-5.586383e+00,-5.586476e+00,-4.761558e+00,-4.761491e+00,-3.018523e+00,-2.435824e+00,-4.331944e+00,-4.328878e+00,-4.629404e+00,-4.628757e+00,-6.545675e-01,-4.792271e-01,-2.106024e+00,-2.105437e+00,0.000000,0.00000,-5.132734e+00,-5.580946e+00,-1.250959e+00,-4.795116e+00,-7.252675e+00,-7.171874e+00,-6.716201e+00,-9.118392e+00,-9.118392e+00,-7.307354e+00,-2.334247e+00,-3.963024e+00,-1.406115e+01,-4.131801e+00,0.000000
25%,-6.333146e-01,-5.380564e-01,-5.379578e-01,-5.273094e-01,-5.271669e-01,-8.267282e-01,-7.783507e-01,-5.555630e-01,-5.494418e-01,-9.437912e-01,-9.434823e-01,-4.341713e-01,-3.101278e-01,-6.847458e-01,-6.842142e-01,0.000000,0.00000,-4.971039e-01,-4.012522e-01,-6.131294e-01,-6.031944e-01,-6.484457e-01,-5.263146e-01,-4.249966e-01,-6.037834e-01,-6.037834e-01,-5.422656e-01,-1.021318e+00,-2.997502e-01,-5.352163e-01,-5.751825e-01,2.000000
50%,-5.219825e-01,1.736813e-01,1.739330e-01,1.507963e-01,1.509466e-01,1.194670e-01,-8.351368e-02,1.809769e-01,1.744366e-01,7.844523e-03,9.643695e-03,-3.165978e-01,-2.353941e-01,-4.409217e-01,-4.397311e-01,0.000000,0.00000,2.108053e-01,2.763271e-01,-3.626969e-01,-2.993098e-01,-6.567956e-03,1.710510e-01,2.555005e-01,1.259313e-01,1.259313e-01,1.855644e-01,7.592589e-02,-1.425072e-01,8.226041e-02,-3.616680e-02,3.000000
75%,8.080490e-01,6.857244e-01,6.857684e-01,6.870790e-01,6.871749e-01,7.778112e-01,7.151016e-01,6.992067e-01,7.039991e-01,7.427266e-01,7.470828e-01,-2.466449e-01,-1.834318e-01,7.407203e-01,7.462227e-01,0.000000,0.00000,6.825232e-01,6.810956e-01,3.357852e-01,3.606306e-01,6.557409e-01,6.775922e-01,6.860858e-01,7.128498e-01,7.128498e-01,6.834608e-01,9.229595e-01,1.162461e-01,6.607770e-01,6.933053e-01,5.000000
max,1.102503e+01,3.866085e+00,3.865987e+00,3.877206e+00,3.877266e+00,3.334075e+00,5.943861e+00,3.668922e+00,3.669393e+00,3.638713e+00,3.641132e+00,1.098381e+01,1.584250e+01,6.377795e+00,6.396067e+00,1.000000,1.00000,3.624593e+00,2.668948e+00,2.709445e+01,2.058265e+01,5.392970e+00,4.535324e+00,3.072890e+00,7.832508e+00,7.832508e+00,6.773694e+00,2.525293e+00,8.611922e+01,3.729718e+00,4.577207e+00,74.000000


## 02) test set 생성

In [ ]:
# 공통전처리가 끝난 test set 불러오기
x_test_J,x_test_B=make_test_set()

# 상관관계가 낮은 변수 제거
x_test_J = x_test_J[['datetime','ru_id']+use_col]
x_test_B = x_test_B[['datetime','ru_id']+use_col]

xs = [x_test_J,x_test_B]

# test set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            standard_scale(df,col)

# train data 합치기
x_test_merge = pd.concat([x_test_J,x_test_B], ignore_index=False)

# index 순으로 정렬
x_test_merge.sort_index(inplace=True)

# datetime, ru_id 제거
x_test_merge_f = x_test_merge.drop(columns=['datetime','ru_id'])

In [ ]:
x_test_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,-0.667542,-0.481082,-0.480596,-0.257629,-0.257425,-0.718056,-0.670180,-1.382757,-1.379720,0.010077,0.011238,-0.226592,-0.171584,1.982971,1.987162,0,0,-1.562210,-1.751742,-0.410825,-0.523185,-1.070125,-0.602207,-0.271383,0.375598,0.375598,-0.332304,-0.859361,-0.318702,-0.011695,-0.577303
1,-0.427040,-1.454413,-1.453854,-1.349899,-1.349338,-1.000441,-0.916495,-1.413417,-1.410143,-0.469494,-0.468596,-0.128414,-0.086296,-0.447929,-0.447758,0,0,-1.509133,-1.624831,-0.469801,-0.633370,0.278680,-1.920447,-1.815504,-1.467930,-1.467930,-1.201107,-1.020477,-0.298595,-1.559757,-1.002757
2,-0.667542,-1.004315,-1.003808,-1.144738,-1.144495,-0.718056,-0.670180,-0.328479,-0.324267,0.010077,0.011238,-0.226592,-0.171584,-0.323645,-0.322739,0,0,0.529450,-0.234396,-0.410825,-0.523185,2.216891,0.324410,-0.267118,-0.652779,-0.652779,0.489501,-0.919680,-0.415243,0.862119,-0.577303
3,-0.427040,0.447827,0.448464,0.530739,0.531318,-1.000441,-0.916495,0.464955,0.470439,-0.469494,-0.468596,-0.128414,-0.086296,-0.447929,-0.447758,0,0,0.132145,-0.138808,-0.469801,-0.633370,-0.835845,-0.060003,-0.204914,0.089064,0.089064,0.278165,-1.084683,-0.298595,-0.750170,0.398415
4,-0.667542,-0.716359,-0.715864,-0.490970,-0.490755,-0.718056,-0.670180,-0.916231,-0.912675,0.573040,0.574515,-0.226592,-0.171584,-0.323645,-0.322739,0,0,0.251629,-0.391863,-0.410825,-0.523185,-0.193587,-0.107811,-0.508853,-0.065758,-0.065758,-0.506874,-0.984066,-0.415243,0.586177,-0.577303


In [ ]:
x_test_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34362 entries, 0 to 34361
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scgfail                       34362 non-null  float64
 1   erabaddatt                    34362 non-null  float64
 2   erabaddsucc                   34362 non-null  float64
 3   endcaddatt                    34362 non-null  float64
 4   endcaddsucc                   34362 non-null  float64
 5   endcmodbymenbatt              34362 non-null  float64
 6   endcmodbymenbsucc             34362 non-null  float64
 7   endcmodbysgnbatt              34362 non-null  float64
 8   endcmodbysgnbsucc             34362 non-null  float64
 9   connestabatt                  34362 non-null  float64
 10  connestabsucc                 34362 non-null  float64
 11  redirectiontolte_coverageout  34362 non-null  float64
 12  redirectiontolte_epsfallback  34362 non-null  float64
 13  h

In [ ]:
x_test_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
count,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,34362.000000,3.436200e+04,3.436200e+04,34362.000000,34362.000000,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04,3.436200e+04
mean,1.654252e-17,-2.365581e-16,1.108349e-16,4.218343e-16,-1.157977e-16,-2.440022e-17,3.432574e-17,-3.639355e-17,2.646804e-17,2.481379e-17,-1.075264e-16,-7.444136e-18,0.000000,-3.308505e-17,2.729516e-17,0.002212,0.002066,-1.162112e-16,-1.852763e-16,3.391217e-17,1.687337e-16,-1.819678e-17,2.770873e-17,4.962757e-18,-3.688983e-16,-3.688983e-16,4.301056e-17,-1.464013e-16,6.663742e-15,-3.639355e-16,4.135631e-19
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015,1.000015e+00,1.000015e+00,0.046978,0.045410,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-6.675420e-01,-3.276950e+00,-3.276468e+00,-3.151740e+00,-3.151197e+00,-1.000441e+00,-9.164945e-01,-2.753031e+00,-2.751333e+00,-9.523143e-01,-9.516885e-01,-2.265920e-01,-0.171584,-4.479294e-01,-4.477579e-01,0.000000,0.000000,-1.562210e+00,-1.803986e+00,-4.698012e-01,-4.716004e+00,-4.138005e+00,-2.451641e+00,-2.052487e+00,-5.315769e+00,-5.315769e+00,-3.146559e+00,-1.930282e+00,-4.152431e-01,-4.744131e+00,-3.055436e+00
25%,-6.675420e-01,-5.431438e-01,-5.425478e-01,-6.225366e-01,-6.219684e-01,-7.180562e-01,-6.701800e-01,-5.852263e-01,-5.813007e-01,-4.694943e-01,-4.685963e-01,-2.265920e-01,-0.171584,-4.479294e-01,-4.477579e-01,0.000000,0.000000,-9.010179e-01,-7.127420e-01,-4.698012e-01,-5.231848e-01,-4.127215e-01,-6.868895e-01,-7.110441e-01,-5.715427e-01,-5.715427e-01,-5.068745e-01,-1.017275e+00,-1.256198e-01,-5.635772e-01,-5.773034e-01
50%,-4.270402e-01,9.514503e-02,9.576755e-02,1.105541e-01,1.107427e-01,-7.180562e-01,-6.701800e-01,1.708896e-01,1.760273e-01,-4.694943e-01,-4.685963e-01,-1.284139e-01,-0.086296,-3.236450e-01,-3.227385e-01,0.000000,0.000000,2.034653e-01,3.375259e-01,-4.108253e-01,-2.578315e-01,5.577489e-02,1.430597e-01,3.168229e-01,1.521601e-01,1.521601e-01,1.977770e-01,4.934158e-02,-2.907865e-02,3.429522e-02,-4.033132e-01
75%,5.177369e-01,7.256316e-01,7.017859e-01,7.319203e-01,7.320817e-01,8.109877e-01,6.581299e-01,7.017032e-01,7.074654e-01,5.730401e-01,5.745145e-01,-1.284139e-01,-0.086296,-3.236450e-01,-3.227385e-01,0.000000,0.000000,8.219134e-01,7.888894e-01,3.655041e-01,1.314463e-01,5.015848e-01,7.736888e-01,7.812339e-01,7.366734e-01,7.366734e-01,6.950006e-01,9.170064e-01,6.746246e-02,6.321676e-01,7.653256e-01
max,1.583369e+01,4.868842e+00,4.869115e+00,4.416785e+00,4.417402e+00,3.844610e+00,6.652412e+00,3.356580e+00,3.346015e+00,5.565503e+00,5.578637e+00,2.399095e+01,12.548270,8.211848e+00,8.224908e+00,1.000000,1.000000,3.985909e+00,2.177397e+00,3.822642e+01,1.724130e+01,4.959684e+00,4.264536e+00,2.300904e+00,5.013328e+00,5.013328e+00,3.958190e+00,2.525293e+00,9.008533e+01,3.351735e+00,7.752723e+00


## 03) data set 저장

In [ ]:
train_merge_f.to_csv('preprocessed_data/submit2_train.csv')
x_test_merge_f.to_csv('preprocessed_data/submit2_test.csv')

# Submit3 : 상관관계 높은 컬럼 사용 / 로그 스케일링 / 로버스트 스케일링

## 01) train set 생성

In [ ]:
# 공통전처리가 끝난 train set 불러오기
x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I = make_train_set()

# 상관관계가 낮은 변수 제거
x_train_A = x_train_A[['datetime',	'ru_id']+use_col]
x_train_C = x_train_C[['datetime',	'ru_id']+use_col]
x_train_D = x_train_D[['datetime',	'ru_id']+use_col]
x_train_E = x_train_E[['datetime',	'ru_id']+use_col]
x_train_F = x_train_F[['datetime',	'ru_id']+use_col]
x_train_G = x_train_G[['datetime',	'ru_id']+use_col]
x_train_H = x_train_H[['datetime',	'ru_id']+use_col]
x_train_I = x_train_I[['datetime',	'ru_id']+use_col]

xs = [x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I]
ys = [y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I]

# train set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # robust scailing
    for col in list(df)[2:]:
        rs(df,col)

# train data 합치기
x_train_merge = x_train_A.copy()
y_train_merge = y_train_A.copy()
for i in range(1,8):
    x_train_merge = pd.concat([x_train_merge, xs[i]], ignore_index=False)
    y_train_merge = pd.concat([y_train_merge, ys[i]], ignore_index=False)

# index 순으로 정렬
x_train_merge.sort_index(inplace=True)
y_train_merge.sort_index(inplace=True)

# AutoML input data type으로 변경
train_merge = pd.concat([x_train_merge, y_train_merge[['uenomax']]], axis=1)

# datetime, ru_id 제거
train_merge_f = train_merge.drop(columns=['datetime','ru_id'])

In [ ]:
train_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,0.0,-0.884523,-0.884523,-0.206003,-0.206003,-1.482307,-1.000000,-2.734545,-2.734545,-0.306270,-0.306270,0.0,0.0,-1.160964,-1.160964,0.0,0.0,-3.670964,-4.399198,-1.095376,-0.718677,-0.400000,-2.629482,-3.209006,-0.865209,-0.865209,-1.090766,-0.542940,-0.7,-0.818182,-0.500000,1
1,0.0,0.103093,0.103093,0.438540,0.438540,1.014162,1.261860,0.773047,0.765527,-0.834044,-0.834044,0.0,0.0,-0.630930,-0.630930,0.0,0.0,0.026671,0.284699,0.000000,0.199207,-0.666667,-0.193652,0.305302,0.751982,0.751982,0.147612,-0.544025,-0.4,-0.533333,0.834044,5
2,1.0,-0.603499,-0.603499,-0.542636,-0.542636,0.417311,0.442507,-0.833803,-0.833803,-1.571069,-1.571069,1.0,0.0,-1.261860,-1.261860,0.0,0.0,0.459742,0.172017,0.383373,0.093114,1.000000,0.474570,0.071059,-0.383695,-0.383695,-0.572062,-0.571689,-0.6,-0.300000,-0.602888,6
3,0.0,-0.912489,-0.912489,-0.834044,-0.834044,0.000000,0.000000,-1.630930,-1.464974,0.630930,0.630930,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.353912,-0.439983,0.000000,0.000000,-0.714286,-0.526745,-0.585510,1.576382,1.576382,-0.439174,-0.466526,-2.0,0.758621,0.000000,1
4,0.0,0.528462,0.528462,0.695977,0.695977,1.000000,1.000000,0.460209,0.460209,-0.553295,-0.553295,0.0,0.0,1.000000,1.000000,0.0,0.0,0.041314,0.524792,0.000000,0.498862,-0.400000,-0.020114,0.533280,0.324553,0.324553,0.489374,-0.555705,-1.0,1.500000,0.684582,4


In [ ]:
train_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137445 entries, 0 to 137444
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   scgfail                       137445 non-null  float64
 1   erabaddatt                    137445 non-null  float64
 2   erabaddsucc                   137445 non-null  float64
 3   endcaddatt                    137445 non-null  float64
 4   endcaddsucc                   137445 non-null  float64
 5   endcmodbymenbatt              137445 non-null  float64
 6   endcmodbymenbsucc             137445 non-null  float64
 7   endcmodbysgnbatt              137445 non-null  float64
 8   endcmodbysgnbsucc             137445 non-null  float64
 9   connestabatt                  137445 non-null  float64
 10  connestabsucc                 137445 non-null  float64
 11  redirectiontolte_coverageout  137445 non-null  float64
 12  redirectiontolte_epsfallback  137445 non-nul

In [ ]:
train_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.00000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000
mean,0.299624,-0.144276,-0.144512,-0.118990,-0.119100,0.065033,0.183116,-0.130388,-0.111136,-0.007109,-0.008542,0.138469,0.061275,0.145672,0.144177,0.018014,0.01696,-0.181590,-0.261924,0.423848,0.415276,0.002025,-0.143599,-0.231703,-0.097182,-0.097182,-0.130160,-0.039127,0.363417,-0.076049,-0.016839,3.718826
std,0.571980,0.842215,0.842211,0.826808,0.826810,0.768003,0.764685,0.819592,0.826199,0.678411,0.678232,0.356496,0.218311,0.744928,0.744118,0.133004,0.12912,0.880223,0.975337,1.792853,1.539683,0.810251,0.846358,0.926386,0.777697,0.777697,0.802907,0.514753,3.041439,0.864313,0.744141,2.781951
min,0.000000,-5.291155,-5.291155,-4.410832,-4.410832,-3.159342,-1.955449,-4.329116,-4.329116,-3.395024,-3.395024,0.000000,0.000000,-1.892789,-1.892789,0.000000,0.00000,-5.913173,-6.788569,-1.095376,-10.871405,-6.000000,-7.517389,-7.531235,-7.502023,-7.502023,-5.880684,-1.306985,-12.000000,-11.600000,-3.631587,0.000000
25%,0.000000,-0.575717,-0.575717,-0.553295,-0.553295,-0.429303,-0.369070,-0.552494,-0.533484,-0.528321,-0.528321,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-0.599128,-0.626282,-0.363274,-0.427125,-0.500000,-0.579551,-0.607317,-0.558218,-0.558218,-0.552989,-0.563425,-0.500000,-0.533333,-0.500000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.693147,0.424283,0.424283,0.446705,0.446705,0.493989,0.630930,0.427487,0.442623,0.490616,0.490616,0.000000,0.000000,0.369070,0.369070,0.000000,0.00000,0.394026,0.365061,0.721532,0.731526,0.500000,0.421285,0.383161,0.440389,0.440389,0.405576,0.435870,0.600000,0.478261,0.457650,5.000000
max,8.228819,3.515885,3.515885,3.204572,3.204572,2.814105,4.906891,3.307742,3.301398,2.477030,2.477030,3.044522,2.484907,5.882643,5.857981,1.000000,1.00000,3.805869,2.869491,42.566164,42.276594,4.750000,4.130537,3.049805,6.262809,6.262809,5.543041,1.293164,462.000000,3.142857,3.481816,74.000000


## 02) test set 생성

In [ ]:
# 공통전처리가 끝난 test set 불러오기
x_test_J,x_test_B=make_test_set()

# 상관관계가 낮은 변수 제거
x_test_J = x_test_J[['datetime','ru_id']+use_col]
x_test_B = x_test_B[['datetime','ru_id']+use_col]

xs = [x_test_J,x_test_B]

# test set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # robust scailing
    for col in list(df)[2:]:
        rs(df,col)

# train data 합치기
x_test_merge = pd.concat([x_test_J,x_test_B], ignore_index=False)

# index 순으로 정렬
x_test_merge.sort_index(inplace=True)

# datetime, ru_id 제거
x_test_merge_f = x_test_merge.drop(columns=['datetime','ru_id'])

In [ ]:
x_test_merge_f.head()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,0.0,-0.369070,-0.369070,-0.244077,-0.244077,0.000000,0.00000,-1.062875,-1.140314,0.00000,0.00000,0.0,0.0,0.693147,0.693147,0.0,0.0,-1.071590,-1.365561,0.0,0.000000,-1.666667,-0.491706,-0.361872,0.187483,0.187483,-0.337180,-0.466526,-1.5,-0.035714,0.000000
1,0.0,-1.298091,-1.298091,-1.362611,-1.362611,-0.710382,-0.63093,-1.247567,-1.247567,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.985529,-1.334135,0.0,-0.513164,0.200000,-1.451921,-1.475730,-1.390743,-1.390743,-1.185885,-0.555705,-1.0,-1.428571,-0.876759
2,0.0,-0.738140,-0.738140,-0.832160,-0.832160,0.000000,0.00000,-0.285471,-0.306270,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.197840,-0.360217,0.0,0.000000,3.333333,0.130214,-0.359057,-0.507757,-0.507757,0.328393,-0.497886,-2.0,0.642857,0.000000
3,0.0,0.201181,0.201181,0.310133,0.310133,-0.710382,-0.63093,0.231563,0.231563,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.041050,-0.338622,0.0,-0.513164,-0.800000,-0.152850,-0.379105,-0.054164,-0.054164,0.061810,-0.588724,-1.0,-0.714286,0.223472
4,0.0,-0.535026,-0.535026,-0.398763,-0.398763,0.000000,0.00000,-0.718868,-0.771244,0.36907,0.36907,0.0,0.0,0.000000,0.000000,0.0,0.0,0.029231,-0.464549,0.0,0.000000,-0.333333,-0.159881,-0.518627,-0.110899,-0.110899,-0.478564,-0.531360,-2.0,0.428571,0.000000


In [ ]:
x_test_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34362 entries, 0 to 34361
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scgfail                       34362 non-null  float64
 1   erabaddatt                    34362 non-null  float64
 2   erabaddsucc                   34362 non-null  float64
 3   endcaddatt                    34362 non-null  float64
 4   endcaddsucc                   34362 non-null  float64
 5   endcmodbymenbatt              34362 non-null  float64
 6   endcmodbymenbsucc             34362 non-null  float64
 7   endcmodbysgnbatt              34362 non-null  float64
 8   endcmodbysgnbsucc             34362 non-null  float64
 9   connestabatt                  34362 non-null  float64
 10  connestabsucc                 34362 non-null  float64
 11  redirectiontolte_coverageout  34362 non-null  float64
 12  redirectiontolte_epsfallback  34362 non-null  float64
 13  h

In [ ]:
x_test_merge_f.describe()

,scgfail,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,airmaculbyte,airmacdlbyte,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
count,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000
mean,0.357134,-0.090755,-0.091164,-0.117612,-0.117932,0.176287,0.274081,-0.088913,-0.094239,0.109125,0.108305,0.023741,0.013417,0.147173,0.146856,0.002212,0.002066,-0.120288,-0.225273,0.537420,0.575763,-0.044452,-0.099237,-0.211157,-0.098531,-0.098531,-0.120430,-0.025327,0.086263,-0.039527,0.125347
std,0.678934,0.750406,0.750409,0.785672,0.785690,0.719087,0.838346,0.764204,0.789628,0.585653,0.585040,0.129502,0.099179,0.380062,0.379713,0.046978,0.045410,0.591404,0.666573,1.222749,1.466411,1.248811,0.684960,0.671186,0.773323,0.773323,0.828512,0.517128,4.390101,0.831219,0.726569
min,0.000000,-2.734545,-2.734545,-2.965268,-2.965268,-0.710382,-0.630930,-2.302450,-2.302450,-0.630930,-0.630930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.071590,-1.454154,0.000000,-6.091977,-6.333333,-1.814901,-1.606101,-3.660198,-3.660198,-2.826780,-1.023585,-2.000000,-4.238095,-2.488568
25%,0.000000,-0.535026,-0.535026,-0.588083,-0.588083,0.000000,0.000000,-0.474792,-0.509384,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.635587,-0.699786,0.000000,0.000000,-0.666667,-0.566771,-0.684716,-0.532394,-0.532394,-0.586610,-0.550988,-0.500000,-0.500000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.693147,0.464974,0.464974,0.438718,0.438718,0.462921,0.630930,0.457298,0.472874,0.369070,0.369070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365721,0.298717,1.000000,0.949914,0.400000,0.430512,0.311929,0.467606,0.467606,0.460917,0.448731,0.333333,0.476190,0.511578
max,7.515700,3.404578,3.404578,3.766597,3.766597,3.244849,6.523562,2.431818,2.594073,2.890372,2.890372,2.302585,1.609438,2.890372,2.890372,1.000000,1.000000,2.295569,1.237759,48.191050,23.912116,7.000000,2.774718,1.336106,4.173001,4.173001,3.137657,1.293164,463.500000,2.904762,4.906891


## 03) data set 저장

In [ ]:
train_merge_f.to_csv('preprocessed_data/submit3_train.csv')
x_test_merge_f.to_csv('preprocessed_data/submit3_test.csv')

# Submit4 : 모든 컬럼 사용 / 로그 스케일링 / 로버스트 스케일링

In [ ]:
# 사용할 컬럼 설정
drop_col_train = ['redirectiontolte_emergencyfallback']
drop_col_test = ['Unnamed: 0','redirectiontolte_emergencyfallback']
log_col_tmp = ['scgfail', 'scgfailratio', 'erabaddatt', 'erabaddsucc', 'endcaddatt', 'endcaddsucc', 'endcmodbymenbatt', 'endcmodbymenbatt', 'endcmodbymenbsucc', 'endcmodbysgnbatt', 'endcmodbysgnbsucc', 'connestabatt', 'connestabsucc', 'redirectiontolte_coverageout', 'redirectiontolte_epsfallback', 'handoveratt', 'handoversucc', 'rlculbyte', 'rlcdlbyte', 'totprbulavg', 'totprbdlavg', 'airmaculbyte', 'airmacdlbyte', 'bler_ul', 'bler_dl', 'rachpreamblea', 'numrar', 'nummsg3', 'endcrelbymenb']

## 01) train set 생성

In [ ]:
# 공통전처리가 끝난 train set 불러오기
x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I = make_train_set()

# 단변량 feature 제거
x_train_A = x_train_A.drop(columns=drop_col_train)
x_train_C = x_train_C.drop(columns=drop_col_train)
x_train_D = x_train_D.drop(columns=drop_col_train)
x_train_E = x_train_E.drop(columns=drop_col_train)
x_train_F = x_train_F.drop(columns=drop_col_train)
x_train_G = x_train_G.drop(columns=drop_col_train)
x_train_H = x_train_H.drop(columns=drop_col_train)
x_train_I = x_train_I.drop(columns=drop_col_train)

xs = [x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I]
ys = [y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I]

# train set별 전처리
for df in xs:
    # log scaling
    for col in log_col_tmp:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # robust scailing
    for col in list(df)[2:]:
        rs(df,col)

# train data 합치기
x_train_merge = x_train_A.copy()
y_train_merge = y_train_A.copy()
for i in range(1,8):
    x_train_merge = pd.concat([x_train_merge, xs[i]], ignore_index=False)
    y_train_merge = pd.concat([y_train_merge, ys[i]], ignore_index=False)

# index 순으로 정렬
x_train_merge.sort_index(inplace=True)
y_train_merge.sort_index(inplace=True)

# AutoML input data type으로 변경
train_merge = pd.concat([x_train_merge, y_train_merge[['uenomax']]], axis=1)

# datetime, ru_id 제거
train_merge_f = train_merge.drop(columns=['datetime','ru_id'])

In [ ]:
train_merge_f.head()

,scgfail,scgfailratio,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,dltransmittedmcsavg,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,0.0,0.000000,-0.884523,-0.884523,-0.206003,-0.206003,-1.482307,-1.000000,-2.734545,-2.734545,-0.306270,-0.306270,0.0,0.0,-1.160964,-1.160964,0.0,0.0,-3.670964,-4.399198,-1.095376,-0.718677,-0.400000,-1.483871,-2.629482,-3.209006,-1.887552,1.659999,-0.865209,-0.865209,-1.090766,-0.542940,-0.7,-0.818182,-0.500000,1
1,0.0,0.000000,0.103093,0.103093,0.438540,0.438540,1.014162,1.261860,0.773047,0.765527,-0.834044,-0.834044,0.0,0.0,-0.630930,-0.630930,0.0,0.0,0.026671,0.284699,0.000000,0.199207,-0.666667,0.548387,-0.193652,0.305302,0.394987,0.304229,0.751982,0.751982,0.147612,-0.544025,-0.4,-0.533333,0.834044,5
2,1.0,1.503043,-0.603499,-0.603499,-0.542636,-0.542636,0.417311,0.442507,-0.833803,-0.833803,-1.571069,-1.571069,1.0,0.0,-1.261860,-1.261860,0.0,0.0,0.459742,0.172017,0.383373,0.093114,1.000000,-0.321429,0.474570,0.071059,-0.600397,-0.400374,-0.383695,-0.383695,-0.572062,-0.571689,-0.6,-0.300000,-0.602888,6
3,0.0,0.000000,-0.912489,-0.912489,-0.834044,-0.834044,0.000000,0.000000,-1.630930,-1.464974,0.630930,0.630930,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.353912,-0.439983,0.000000,0.000000,-0.714286,-0.220000,-0.526745,-0.585510,-1.176345,-0.133592,1.576382,1.576382,-0.439174,-0.466526,-2.0,0.758621,0.000000,1
4,0.0,0.000000,0.528462,0.528462,0.695977,0.695977,1.000000,1.000000,0.460209,0.460209,-0.553295,-0.553295,0.0,0.0,1.000000,1.000000,0.0,0.0,0.041314,0.524792,0.000000,0.498862,-0.400000,0.978261,-0.020114,0.533280,0.857415,0.107598,0.324553,0.324553,0.489374,-0.555705,-1.0,1.500000,0.684582,4


In [ ]:
train_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137445 entries, 0 to 137444
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   scgfail                       137445 non-null  float64
 1   scgfailratio                  137445 non-null  float64
 2   erabaddatt                    137445 non-null  float64
 3   erabaddsucc                   137445 non-null  float64
 4   endcaddatt                    137445 non-null  float64
 5   endcaddsucc                   137445 non-null  float64
 6   endcmodbymenbatt              137445 non-null  float64
 7   endcmodbymenbsucc             137445 non-null  float64
 8   endcmodbysgnbatt              137445 non-null  float64
 9   endcmodbysgnbsucc             137445 non-null  float64
 10  connestabatt                  137445 non-null  float64
 11  connestabsucc                 137445 non-null  float64
 12  redirectiontolte_coverageout  137445 non-nul

In [ ]:
train_merge_f.describe()

,scgfail,scgfailratio,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,dltransmittedmcsavg,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.00000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000,137445.000000
mean,0.299624,0.421819,-0.144276,-0.144512,-0.118990,-0.119100,0.065033,0.183116,-0.130388,-0.111136,-0.007109,-0.008542,0.138469,0.061275,0.145672,0.144177,0.018014,0.01696,-0.181590,-0.261924,0.423848,0.415276,0.002025,0.046373,-0.143599,-0.231703,-0.107005,-0.063540,-0.097182,-0.097182,-0.130160,-0.039127,0.363417,-0.076049,-0.016839,3.718826
std,0.571980,0.809301,0.842215,0.842211,0.826808,0.826810,0.768003,0.764685,0.819592,0.826199,0.678411,0.678232,0.356496,0.218311,0.744928,0.744118,0.133004,0.12912,0.880223,0.975337,1.792853,1.539683,0.810251,0.761797,0.846358,0.926386,1.790549,0.919867,0.777697,0.777697,0.802907,0.514753,3.041439,0.864313,0.744141,2.781951
min,0.000000,0.000000,-5.291155,-5.291155,-4.410832,-4.410832,-3.159342,-1.955449,-4.329116,-4.329116,-3.395024,-3.395024,0.000000,0.000000,-1.892789,-1.892789,0.000000,0.00000,-5.913173,-6.788569,-1.095376,-10.871405,-6.000000,-2.250000,-7.517389,-7.531235,-18.786242,-4.973974,-7.502023,-7.502023,-5.880684,-1.306985,-12.000000,-11.600000,-3.631587,0.000000
25%,0.000000,0.000000,-0.575717,-0.575717,-0.553295,-0.553295,-0.429303,-0.369070,-0.552494,-0.533484,-0.528321,-0.528321,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-0.599128,-0.626282,-0.363274,-0.427125,-0.500000,-0.483871,-0.579551,-0.607317,-0.482096,-0.538371,-0.558218,-0.558218,-0.552989,-0.563425,-0.500000,-0.533333,-0.500000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.693147,0.759174,0.424283,0.424283,0.446705,0.446705,0.493989,0.630930,0.427487,0.442623,0.490616,0.490616,0.000000,0.000000,0.369070,0.369070,0.000000,0.00000,0.394026,0.365061,0.721532,0.731526,0.500000,0.500000,0.421285,0.383161,0.517904,0.454188,0.440389,0.440389,0.405576,0.435870,0.600000,0.478261,0.457650,5.000000
max,8.228819,5.303305,3.515885,3.515885,3.204572,3.204572,2.814105,4.906891,3.307742,3.301398,2.477030,2.477030,3.044522,2.484907,5.882643,5.857981,1.000000,1.00000,3.805869,2.869491,42.566164,42.276594,4.750000,5.750000,4.130537,3.049805,14.060587,5.686567,6.262809,6.262809,5.543041,1.293164,462.000000,3.142857,3.481816,74.000000


## 02) test set 생성

In [ ]:
# 공통전처리가 끝난 test set 불러오기
x_test_J,x_test_B=make_test_set()

# 단변량 feature 제거
x_test_J = x_test_J.drop(columns=drop_col_test)
x_test_B = x_test_B.drop(columns=drop_col_test)

xs = [x_test_J,x_test_B]

# test set별 전처리
for df in xs:
    # log scaling
    for col in log_col_tmp:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # robust scailing
    for col in list(df)[2:]:
        rs(df,col)

# train data 합치기
x_test_merge = pd.concat([x_test_J,x_test_B], ignore_index=False)

# index 순으로 정렬
x_test_merge.sort_index(inplace=True)

# datetime, ru_id 제거
x_test_merge_f = x_test_merge.drop(columns=['datetime','ru_id'])

In [ ]:
x_test_merge_f.head()

,scgfail,scgfailratio,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,dltransmittedmcsavg,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,0.0,0.0,-0.369070,-0.369070,-0.244077,-0.244077,0.000000,0.00000,-1.062875,-1.140314,0.00000,0.00000,0.0,0.0,0.693147,0.693147,0.0,0.0,-1.071590,-1.365561,0.0,0.000000,-1.666667,0.038462,-0.491706,-0.361872,-0.345020,0.743185,0.187483,0.187483,-0.337180,-0.466526,-1.5,-0.035714,0.000000
1,0.0,0.0,-1.298091,-1.298091,-1.362611,-1.362611,-0.710382,-0.63093,-1.247567,-1.247567,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.985529,-1.334135,0.0,-0.513164,0.200000,-0.815385,-1.451921,-1.475730,-6.045324,-1.689675,-1.390743,-1.390743,-1.185885,-0.555705,-1.0,-1.428571,-0.876759
2,0.0,0.0,-0.738140,-0.738140,-0.832160,-0.832160,0.000000,0.00000,-0.285471,-0.306270,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.197840,-0.360217,0.0,0.000000,3.333333,-0.615385,0.130214,-0.359057,1.732271,-0.404324,-0.507757,-0.507757,0.328393,-0.497886,-2.0,0.642857,0.000000
3,0.0,0.0,0.201181,0.201181,0.310133,0.310133,-0.710382,-0.63093,0.231563,0.231563,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,-0.041050,-0.338622,0.0,-0.513164,-0.800000,-0.138462,-0.152850,-0.379105,-0.261025,-1.206677,-0.054164,-0.054164,0.061810,-0.588724,-1.0,-0.714286,0.223472
4,0.0,0.0,-0.535026,-0.535026,-0.398763,-0.398763,0.000000,0.00000,-0.718868,-0.771244,0.36907,0.36907,0.0,0.0,0.000000,0.000000,0.0,0.0,0.029231,-0.464549,0.0,0.000000,-0.333333,-0.576923,-0.159881,-0.518627,-0.394908,-0.827277,-0.110899,-0.110899,-0.478564,-0.531360,-2.0,0.428571,0.000000


In [ ]:
x_test_merge_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34362 entries, 0 to 34361
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scgfail                       34362 non-null  float64
 1   scgfailratio                  34362 non-null  float64
 2   erabaddatt                    34362 non-null  float64
 3   erabaddsucc                   34362 non-null  float64
 4   endcaddatt                    34362 non-null  float64
 5   endcaddsucc                   34362 non-null  float64
 6   endcmodbymenbatt              34362 non-null  float64
 7   endcmodbymenbsucc             34362 non-null  float64
 8   endcmodbysgnbatt              34362 non-null  float64
 9   endcmodbysgnbsucc             34362 non-null  float64
 10  connestabatt                  34362 non-null  float64
 11  connestabsucc                 34362 non-null  float64
 12  redirectiontolte_coverageout  34362 non-null  float64
 13  r

In [ ]:
x_test_merge_f.describe()

,scgfail,scgfailratio,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,endcmodbysgnbatt,endcmodbysgnbsucc,connestabatt,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoversucc,reestabatt,reestabsucc,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,dltransmittedmcsavg,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
count,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000,34362.000000
mean,0.357134,0.420228,-0.090755,-0.091164,-0.117612,-0.117932,0.176287,0.274081,-0.088913,-0.094239,0.109125,0.108305,0.023741,0.013417,0.147173,0.146856,0.002212,0.002066,-0.120288,-0.225273,0.537420,0.575763,-0.044452,0.044630,-0.099237,-0.211157,-0.064615,-0.066601,-0.098531,-0.098531,-0.120430,-0.025327,0.086263,-0.039527,0.125347
std,0.678934,0.765590,0.750406,0.750409,0.785672,0.785690,0.719087,0.838346,0.764204,0.789628,0.585653,0.585040,0.129502,0.099179,0.380062,0.379713,0.046978,0.045410,0.591404,0.666573,1.222749,1.466411,1.248811,0.647588,0.684960,0.671186,1.698199,0.917742,0.773323,0.773323,0.828512,0.517128,4.390101,0.831219,0.726569
min,0.000000,0.000000,-2.734545,-2.734545,-2.965268,-2.965268,-0.710382,-0.630930,-2.302450,-2.302450,-0.630930,-0.630930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.071590,-1.454154,0.000000,-6.091977,-6.333333,-0.884615,-1.814901,-1.606101,-6.045324,-1.980672,-3.660198,-3.660198,-2.826780,-1.023585,-2.000000,-4.238095,-2.488568
25%,0.000000,0.000000,-0.535026,-0.535026,-0.588083,-0.588083,0.000000,0.000000,-0.474792,-0.509384,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.635587,-0.699786,0.000000,0.000000,-0.666667,-0.492308,-0.566771,-0.684716,-0.420393,-0.549361,-0.532394,-0.532394,-0.586610,-0.550988,-0.500000,-0.500000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.693147,0.839026,0.464974,0.464974,0.438718,0.438718,0.462921,0.630930,0.457298,0.472874,0.369070,0.369070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365721,0.298717,1.000000,0.949914,0.400000,0.492308,0.430512,0.311929,0.543360,0.436084,0.467606,0.467606,0.460917,0.448731,0.333333,0.476190,0.511578
max,7.515700,4.615121,3.404578,3.404578,3.766597,3.766597,3.244849,6.523562,2.431818,2.594073,2.890372,2.890372,2.302585,1.609438,2.890372,2.890372,1.000000,1.000000,2.295569,1.237759,48.191050,23.912116,7.000000,3.653846,2.774718,1.336106,7.238178,3.376415,4.173001,4.173001,3.137657,1.293164,463.500000,2.904762,4.906891


## 03) data set 저장

In [ ]:
train_merge_f.to_csv('preprocessed_data/submit4_train.csv')
x_test_merge_f.to_csv('preprocessed_data/submit4_test.csv')

# Submit5 : 모든 컬럼 사용 / 로그 스케일링 / 로버스트 스케일링 / 차분 후 이동평균 feature 추가

## 01) train set 생성

In [ ]:
# 공통전처리가 끝난 train set 불러오기
x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I,y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I = make_train_set()

# 상관관계가 낮은 변수 제거
x_train_A = x_train_A[['datetime',	'ru_id']+use_col]
x_train_C = x_train_C[['datetime',	'ru_id']+use_col]
x_train_D = x_train_D[['datetime',	'ru_id']+use_col]
x_train_E = x_train_E[['datetime',	'ru_id']+use_col]
x_train_F = x_train_F[['datetime',	'ru_id']+use_col]
x_train_G = x_train_G[['datetime',	'ru_id']+use_col]
x_train_H = x_train_H[['datetime',	'ru_id']+use_col]
x_train_I = x_train_I[['datetime',	'ru_id']+use_col]

xs = [x_train_A,x_train_C,x_train_D,x_train_E,x_train_F,x_train_G,x_train_H,x_train_I]
ys = [y_train_A,y_train_C,y_train_D,y_train_E,y_train_F,y_train_G,y_train_H,y_train_I]

# train set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 이상치 제거 및 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            remov_outlier(df,col)
            standard_scale(df,col)

# train data 합치기
x_train_merge = x_train_A.copy()
y_train_merge = y_train_A.copy()
for i in range(1,8):
    x_train_merge = pd.concat([x_train_merge, xs[i]], ignore_index=False)
    y_train_merge = pd.concat([y_train_merge, ys[i]], ignore_index=False)

# index 순으로 정렬
x_train_merge.sort_index(inplace=True)
y_train_merge.sort_index(inplace=True)

# AutoML input data type으로 변경
train_merge = pd.concat([x_train_merge, y_train_merge[['uenomax']]], axis=1)

# datetime, ru_id 제거
train_merge_f = train_merge.drop(columns=['datetime','ru_id'])

# 결측치가 있는 행 제거
train_merge_f = train_merge_f.dropna()

### 계절차분, 이동평균모형 생성

In [ ]:
# 8씩 계절차분 진행
seasonal_diff = train_merge_f['uenomax'].diff(periods=8)

# 계절 차분 값을 새로운 변수로 추가
train_merge_f['seasonal_diff'] = seasonal_diff


# 이동평균 모형(MA) 적용
# 이동평균을 계산할 때 사용할 윈도우 크기를 12로 설정. 5분 간격의 데이터를 1시간 단위로 묶기 위해 사용.
window_size_hours = 12  # 5분을 시간 단위로 환산 (1/12 시간)
train_merge_f['moving_average'] = train_merge_f['seasonal_diff'].rolling(window=window_size_hours).mean()

# NaN 값을 제거
train_merge_f = train_merge_f.dropna()

## 02) test set 생성

In [ ]:
# 공통전처리가 끝난 test set 불러오기
x_test_J,x_test_B=make_test_set()

# 상관관계가 낮은 변수 제거
x_test_J = x_test_J[['datetime','ru_id']+use_col]
x_test_B = x_test_B[['datetime','ru_id']+use_col]

xs = [x_test_J,x_test_B]

# test set별 전처리
for df in xs:
    # log scaling
    for col in log_col:
        log_scale(df,col)
    # 범주형 데이터 수정
    df.loc[df['reestabatt']!=0,'reestabatt'] = 1
    df.loc[df['reestabsucc']!=0,'reestabsucc'] = 1
    # 정규화
    for col in list(df)[2:]:
        if not col.startswith('ree'): # datetime, ru_id, 범주형 변수 2개 제외
            standard_scale(df,col)

# train data 합치기
x_test_merge = pd.concat([x_test_J,x_test_B], ignore_index=False)

# index 순으로 정렬
x_test_merge.sort_index(inplace=True)

# datetime, ru_id 제거
x_test_merge_f = x_test_merge.drop(columns=['datetime','ru_id'])

## 03) data set 저장

In [ ]:
train_merge_f.to_csv('preprocessed_data/submit5_train.csv')
x_test_merge_f.to_csv('preprocessed_data/submit5_test.csv')